Idea and Hypothesis:
When evolving the base CTRNN state and the HP mechanism together to perform well at various temperature values (arbitrary global perturbation), we expect two non-mutually exclusive strategies to emerge

1) Move the base state to an area of parameter space where temperature variation induces a relatively mild fitness dropoff (is aligned with a degenerate fitness ridge)

2) Identify an HP mechanism which activates in response to temperature changes and restores parameters to a fit location

2b) Identify an HP mechanism which, when seeded with the appropriate base state, drives an oscillation that is reasonably fit and maintains this oscillation despite temperature variation. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from CTRNNclass import *
from acceptance import *
from HPevolution import *
from pyloricfitness import *
from SampleGenerators import *

### Additional thoughts:
- How do these things depend on the nature of the perturbation? 

    - Variations to test: (1) Ordered global direciton vs. random noise (makes impossible to align to degenerate edge), (2) Different ordered global directions may be easier or harder to find edges corresponding to them and/or HP mechanisms suited to them and/or HP mechanisms which can continue to drive appropriate oscillations under their conditions
    
- How do these things depend on the nature of the task?

    - For example, with the walker, maybe degenerate ridges are easier to find, or maybe HP mechanisms are easier to find, or maybe this HP formulation is just better suited to the task (i.e. sliding window can make the shape, order doesn't matter) 
    
    - What about a behaving circuit that needs to respond to stimuli appropriately (not a CPG)? Then, maybe HP is a less viable option (disrupts appropriate responses). Maybe this could be remedied with slowing it down (separation of timescales), but how much is reasonable? And if not, this would be a major hit to the idea of HP because true CPG's do not exist. 
    
- How do these things depend on the nature of HP?

    - Differential application, sliding window, Hywell Williams vs. DiPaolo protocols, ...
    
    - Including a pacemaker element?
    
    - Dealing with time constants - should they reasonably be affected by temperature? should they reasonably be under the control of HP? What would it mean for animals if their HP had or didn't have control over them?
    
    - Demonstrated that HP needs capping so that weights don't go to zero with biases exploding - think about why that is

- How do these things depend on the details of evolution?

    - Evolving basal state first and then HP (or vice versa) vs. concurrently 
    
    - Differently weighting the importance of performing under basal state vs. performing under perturbation
    
    - Crossover and mutation differences (keeping basal state and HP as separate units)
    
- How would an experimentalist observe this?

    - Crabs pulled from the environment with variability along the degenerate fitness ridges? Response bias or evolution at work?
        - Different arrangement when the perturbations have been larger/more disruptive? (i.e. climate change)
        - Observe correlational structure even though none is "built in" to the HP mechanisms? Or even when no HP mechanism is present at all
    
    - 

In [2]:
#a pyloric-like neurongenome evolved from the other notebook
#in case ever needed

sol1 = np.array([ 14.82024187,  10.61253584,-12.15141158,  -4.61967513,-15.81543225,  15.62745189,  14.71262429,   2.0523492 , 1.86045721, -10.25813196,  -0.29218389,  -2.47607644, 1.13091869,   4.00518322,   3.57617411])

In [3]:
##### Global perturbation (temperature) params #######
n = 3
freepars = (n**2) + n  #number of pars that will be perturbed by temperature (not time constants)
globalpertdirection = np.random.uniform(-1,1,size=freepars) 
pertvector = (globalpertdirection/np.linalg.norm(globalpertdirection)) #normalized to have length 1
globalpertmagnitude = 1 #length it will actually have in practice
pts = 3 #number of locations to test along the pertvector

print(pertvector)

def fitnessfunc(genome):
    '''genome has the form [weights,biases,timeconsts,lbs,ubs,taub,tauw,slidingwindow]. 
    pertvector and the number of sample points along it is defined outside the function
    Apply perts of different magnitudes to the homeostatic CTRNN and average its pyloric fitness @ each point'''
    neurongenome = genome[:-((2*n)+3)]
    HPgenome = genome[-((2*n)+3):]
    fitnesssum = 0
    for i in np.linspace(0,globalpertmagnitude,pts): 
        neurongenome[:len(pertvector)] += (i*pertvector)
        fitnesssum += pyloriclike(neurongenome,HPgenome,debugging = True)
    return (fitnesssum/pts)

[-0.24274307  0.34562342 -0.40816932 -0.39281511  0.30441172  0.05436264
 -0.05058103 -0.36093516  0.11789112  0.2758798  -0.30641782  0.29725419]


In [4]:
#####Microbial Genetic Algorithm Params ###############
CTRNNsize = 3
popsize = 15
startpopulation = np.concatenate((randomCTRNNsample(CTRNNsize,popsize),randomHPsample(CTRNNsize,popsize)),axis=1)
print('example genome', startpopulation[0])
recombProb = .5 
mutatProb = .25 #really, its the magnitude of mutation
generations = 200
differentialapp = [1,1,1] #HP can apply itself to any neuron

M = Microbial(fitnessfunc, startpopulation, recombProb, mutatProb, generations, differentialapp)

example genome [-1.73778055e+00 -1.10180725e+01 -7.96290769e+00 -1.42584240e+01
 -5.77364841e+00  3.14400917e+00  4.35290129e+00  8.94683184e+00
  1.76295220e+00  3.68958268e+00  1.37906696e+01 -1.18951736e+01
  5.57679786e+00  5.99587088e+00  9.04460841e+00  1.43200512e-01
  2.81091665e-03  4.24693432e-01  5.18232602e-01  5.43353649e-01
  9.30700655e-01  4.20251738e+01  2.02814381e+01  2.00000000e+00]


In [5]:
M.run()
M.showFitness()

#must increase transients for a longer HP process
#doubleperiodicity much more common with HP, or at least the appearance of it from a 3D projection

0
0.7660449275465478


In [8]:
print(M.fitStats())

(0.1605865410764879, 0.7326001666055033, array([-1.66277628e+00,  6.08372573e+00,  2.02550211e-02,  9.42733663e-01,
        8.46697321e+00,  4.99193437e+00, -5.60691553e+00, -6.66479592e+00,
        5.06919151e+00,  3.36623523e+00, -3.18997036e+00, -4.84189892e+00,
        7.37470557e+00,  9.79376709e+00,  2.56164698e+00,  3.58472612e-01,
        2.09458835e-01,  3.42017940e-01,  3.68472612e-01,  2.19458835e-01,
        8.28364889e-01,  3.99910632e+01,  1.90137781e+01,  4.00000000e+01]))
